In [22]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
%matplotlib tk

In [23]:

def initialize_spin_lattice(N):
    return np.random.choice([-1, 1], size=(N, N))

def calculate_energy(spin_lattice):
    energy = 0
    N = len(spin_lattice)
    for i in range(N):
        for j in range(N):
            energy += -spin_lattice[i, j] * (spin_lattice[(i+1)%N, j] + spin_lattice[i, (j+1)%N])
    return energy

def metropolis(spin_lattice, beta):
    N = len(spin_lattice)
    i, j = np.random.randint(N, size=2)
    dE = 2 * spin_lattice[i, j] * (spin_lattice[(i+1)%N, j] + spin_lattice[i, (j+1)%N] +
                                    spin_lattice[i, (j-1)%N] + spin_lattice[(i-1)%N, j])
    if dE < 0 or np.random.rand() < np.exp(-beta * dE):
        spin_lattice[i, j] *= -1
        return 2 * dE
    return 0

def calculate_magnetization(spin_lattice):
    return np.sum(spin_lattice)

def correlation_function(spin_lattice):
    N = len(spin_lattice)
    correlation = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            s = spin_lattice[i, j]
            correlation += s * np.roll(spin_lattice, (i, j), axis=(0, 1))
    return correlation / (N**2)

# Parameters
N = 5  
n_steps = 1000
beta = 1.0

# Initialize lattice
spin_lattice = initialize_spin_lattice(N)

# Lists to store data for plotting
energies = []
magnetizations = []

# Metropolis steps
for step in range(n_steps):
    dE = metropolis(spin_lattice, beta)
    energies.append(calculate_energy(spin_lattice))
    magnetizations.append(calculate_magnetization(spin_lattice))

# Plot energy convergence
plt.figure(figsize=(10, 5))
plt.plot(energies)
plt.title('Energy Convergence')
plt.xlabel('Monte Carlo Steps')
plt.ylabel('Energy')
plt.show()



In [16]:

# Initialize lattice
spin_lattice = initialize_spin_lattice(N)

# Lists to store data for plotting
energies = []
magnetizations = []
spin_configs = []

# Initialize figure for animation
fig, ax = plt.subplots(figsize=(8, 8))

def update(frame):
    ax.clear()
    dE = metropolis(spin_lattice, beta)
    energies.append(calculate_energy(spin_lattice))
    magnetizations.append(calculate_magnetization(spin_lattice))
    spin_configs.append(np.copy(spin_lattice))  # Store current spin configuration
    ax.quiver(np.arange(N), np.arange(N), np.cos(spin_lattice), np.sin(spin_lattice), scale=20, color='r')
    ax.set_title(f'Step {frame+1}')
    return ax

ani = FuncAnimation(fig, update, frames=n_steps, repeat=False)
# Save the animation as a GIF
ani.save('spin_animation.gif', writer='pillow', fps=10)  # Specify the desired file name and frame rate

plt.show()



In [24]:
# Plot the evolution of spin configurations
fig, axs = plt.subplots(1, 3, figsize=(8, 6))
for i, step in enumerate([0, n_steps//2, n_steps-1]):
    if step < len(spin_configs):
        axs[i].quiver(np.arange(N), np.arange(N), np.cos(spin_configs[step]), np.sin(spin_configs[step]), scale=10, color='r')
        axs[i].set_title(f'Step {step+1}')
        axs[i].axis('on')

plt.tight_layout()
plt.show()